In this notebook we will guide you through our solution for the second homework of ADA.

# Part 1: Extracting the options from the dropdown menu

We start with the URL from the [homepage](http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247) where you can the select the different years/semesters etc..

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sys
import numpy as np


urlHome = 'http://isa.epfl.ch/imoniteur_ISAP/\
%21gedpublicreports.htm?ww_i_reportmodel=133685247'

We then use requests and BeautifulSoup to import the HTML data from the homepage.

In [16]:
home = requests.get(urlHome)
homeSoup = BeautifulSoup(home.content,'lxml')

The insepetion tool of our browser helped learn that we were to access a #document that is in the frame of one of the framesets of the HTML page, the document is sourced from another page of IS-Academia that we have to access. To access it, we scroll through the framesets and create a list of all the frames. 

In [17]:
iframe = []
for frame in homeSoup.frameset.find_all('frame'):
    iframe.append(frame)

urlOptions = iframe[1]['src']

print(urlOptions)

!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247


The url of the source page is found, we can use it to get the other HTML page. Again we use request and beautifulsoup 
to import the HTML data from this new page.

In [18]:
urlOptions = 'http://isa.epfl.ch/imoniteur_ISAP/' + urlOptions
options = requests.get(urlOptions)
soupOptions = BeautifulSoup(options.content,'lxml')





From this beautifulsoup object we can create a dictionary containing the name of each form parameters as key and the value of each parameters as value. We do this by scrolling to the option tag which contains this information.

In [19]:
#dictionary with the name and the value of each parameters
dic_name_value = {}

#Loop that passes trough all the option tag
for f in soupOptions.find_all('option'):
    
    #Filling of the dictionary
    dic = {f.string : f.attrs["value"]}
    dic_name_value.update(dic)


    
print(dic_name_value)

{'Microtechnique': '945244', "Semestre d'automne": '2936286', 'Master semestre 1': '2230106', 'EME (EPFL Middle East)': '637841336', 'Bachelor semestre 6b': '2226785', 'Mathématiques': '944590', 'Bachelor semestre 1': '249108', 'Semestre printemps': '2754553', None: 'null', 'Projet Master printemps': '3781783', '2010-2011': '39486325', 'Semestre automne': '953159', 'Génie civil': '942623', '2009-2010': '978195', 'Bachelor semestre 5b': '2226768', 'Projet Master automne': '249127', 'Physique': '945571', 'Mineur semestre 1': '2335667', 'Chimie et génie chimique': '246696', 'Génie mécanique': '944263', 'Bachelor semestre 3': '942155', '2007-2008': '978181', 'Master semestre 4': '2230140', 'Management de la technologie': '946882', 'Section FCUE': '1574548993', 'Architecture': '942293', 'Cours de mathématiques spéciales': '943282', '2015-2016': '213638028', 'Bachelor semestre 2': '249114', 'Master semestre 3': '2230128', 'Mise à niveau': '2063602308', 'Stage printemps master': '2227132', '2

We separate this dictionary into smaller dictionaries specific to each form parameters. Which means one for the section, one for the academic period, etc. We do this specifically for the bachelor students, in order to complete the first task.

In [20]:
#From the dictionnary we created before with the name and the value, we extract the parameters that we want. 
#We create separated dictionary for every form parameters.

#First parameter: Informatique
parameter_section = {'Informatique' : (dic_name_value['Informatique'])}
print(parameter_section)


#Seconde parameter: Semester

#Initialization of the dictionary
parameter_semester = {}



#Loop that will pass trough the whole dictionary and select the ones that start with Semestre
for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith('Semestre d'):
            parameter_semester[key]=value
print(parameter_semester)


#Third parameter: years
parameter_years = {}

for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith('2'):
            parameter_years[key]=value
            
print(parameter_years)


#Fourth parameter: Pedagogical period

test = ["Ba"]   #test = ["Ba", "Ma","Min","Mis","Pro","Sem","Sta"]
parameter_period = {};
for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith(tuple(test)):
            parameter_period[key]=value
print(parameter_period)



{'Informatique': '249847'}
{"Semestre d'automne": '2936286', 'Semestre de printemps': '2936295'}
{'2011-2012': '123455150', '2016-2017': '355925344', '2009-2010': '978195', '2012-2013': '123456101', '2007-2008': '978181', '2010-2011': '39486325', '2014-2015': '213637922', '2013-2014': '213637754', '2015-2016': '213638028', '2008-2009': '978187'}
{'Bachelor semestre 1': '249108', 'Bachelor semestre 2': '249114', 'Bachelor semestre 3': '942155', 'Bachelor semestre 4': '942163', 'Bachelor semestre 6': '942175', 'Bachelor semestre 5b': '2226768', 'Bachelor semestre 6b': '2226785', 'Bachelor semestre 5': '942120'}


From theses dictionaries, we load the data directly from the IS-academia servers using pd.read_html. We use the standard url and we change the form parameters values for each values in each dictionaries using the %s method.

In [21]:
# Now we want to request the is-academia servers for the specific data we need, using the dictionaries that we created
#before. We will use for loop passing trough every dictionaries in order to compose the multiple requests needed.

data_all = pd.DataFrame()

#Multiple loop that permits to use each values in each dictionaries
for value_section in parameter_section.values():
    for value_years in parameter_years.values():
        for value_period in parameter_period.values():
            for value_semester in parameter_semester.values():
                
                #We use a try to prevent the problem when there is a empty list.
                try:
                    #The url is changed for every values in each loops
                    student_data = pd.read_html('http://isa.epfl.ch/imoniteur_ISAP//!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=%s&ww_x_PERIODE_ACAD=%s&ww_x_PERIODE_PEDAGO=%s&ww_x_HIVERETE=%s' % (value_section, value_years, value_period, value_semester))
                    
                    #The data is reshaped to be more suite for subsequent analysis
                    student_data.columns = student_data.loc[[1]].values.reshape(12,)
                    student_data.drop(student_data.index[[0,1]],inplace=True)
                    student_data = student_data[['Civilité','Nom Prénom',]]
                except:
                    pass
                
                #Each block of data is concatenated to have a unique dataframe
                data_all = pd.concat([data_all, student_data[0]], axis = 0)
                
data_all



,0,1,2,3,4,5,6,7,8,9,10,11
0,"Informatique, 2011-2012, Bachelor semestre 1 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
2,Monsieur,Aiulfi Loris Sandro,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,202293,NaN
3,Monsieur,Akiba David,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,206418,NaN
4,Monsieur,Albasini Romain,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,198197,NaN
5,Monsieur,Albrecht Pablo,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,212726,NaN
6,Monsieur,Alonso Seisdedos Florian,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,215576,NaN
7,Monsieur,Amirghasemi Nicolas Cyrus René,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,213361,NaN
8,Monsieur,Amorim Afonso Caldeira Da Silva Pedro Maria,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,213618,NaN
9,Monsieur,Andreina Sébastien Laurent,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,215623,NaN


# Parameter Generation for the Master Students (TASK 2)

In [432]:
#parameters Period Master :Only the period (Bachelor to Master will be changed)

test = ["Mas"]   #test = ["Ba", "Ma","Min","Mis","Pro","Sem","Sta"]
parameter_period_Master = {};
for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith(tuple(test)):
            parameter_period_Master[key]=value
print(parameter_period_Master)


  

{'Master semestre 4': '2230140', 'Master semestre 2': '942192', 'Master semestre 3': '2230128', 'Master semestre 1': '2230106'}


# Drilling Down for Data (Master)

In [464]:
data_master = pd.DataFrame()
for value_section in parameter_section.values():
    for value_years in parameter_years.values():
        for value_period in parameter_period_Master.values():
            for value_semester in parameter_semester.values():
                  #We use a try to prevent the problem when there is a empty list.
                try:
                    #The url is changed for every values in each loops
                    student_data = pd.read_html('http://isa.epfl.ch/imoniteur_ISAP//!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=%s&ww_x_PERIODE_ACAD=%s&ww_x_PERIODE_PEDAGO=%s&ww_x_HIVERETE=%s' % (value_section, value_years, value_period, value_semester),header=None)
                    
                    #The data is reshaped to be more suite for subsequent analysis
                    student_data.columns = student_data.loc[[1]].values.reshape(12,)
                    student_data.drop(student_data.index[[0,1]],inplace=True)
                    student_data = student_data[['Civilité','Nom Prénom',]]
                except:
                    pass
                
                #Each block of data is concatenated to have a unique dataframe
                data_master = pd.concat([data_master, student_data[0]], axis = 0)# we could use keys here
                


In [465]:
data_master.head() #view data

,0,1,2,3,4,5,6,7,8,9,10,11
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
2,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
3,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
4,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN


In [466]:
# Lets describe the data

data_master.columns =  ["Civilité","Nom Prénom","Orientation Bachelor" ,"Orientation Master","Specialisation","Filière opt.","Mineur","Statut","Type Echange","Ecole Echange","No Sciper","NaN"]

In [467]:
data_master.head()

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
2,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
3,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
4,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN


In [468]:
#fix the column head 
data_master=data_master.drop(data_master.index[[1]])
data_master

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
3,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
4,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN
5,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,172264,NaN
0,"Informatique, 2011-2012, Projet Master printem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Monsieur,Bloch Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,178553,NaN
3,Monsieur,Bricola Jean-Charles,NaN,NaN,"Signals, Images and Interfaces",NaN,NaN,Stage,NaN,NaN,180731,NaN
4,Monsieur,Chakrabarty Soumitro,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,199654,NaN
5,Monsieur,Dosmukhamedov Diyar,NaN,NaN,Internet computing,NaN,NaN,Présent,NaN,NaN,192861,NaN


In [469]:
#fix index numbering
data_master.index = range(len(data_master.index))


In [470]:
#drop Nan
data_master= data_master.drop(["NaN"], 1)



In [471]:
# drop orientation Bachelor

data_master= data_master.drop(["Orientation Bachelor"], 1)




In [472]:
data_master.head()

,Civilité,Nom Prénom,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027
2,Monsieur,Brutsche Florian,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852
3,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805
4,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,Stage,NaN,NaN,172264


In [473]:
preview = [];
for l in data_master.columns :
        m= data_master[l].value_counts()
        preview.append(m)
data_master_summary= pd.DataFrame(preview[0])

data_master_summary #shows initial value for Monsieur and Madame

,Civilité
Monsieur,264
Madame,36
"Informatique, 2015-2016, Projet Master automne (2 ét.)",3
"Informatique, 2013-2014, Projet Master automne (3 ét.)",3
"Informatique, 2007-2008, Projet Master automne (1 ét.)",3
"Informatique, 2012-2013, Projet Master automne (2 ét.)",3
"Informatique, 2014-2015, Projet Master automne (1 ét.)",3
"Informatique, 2011-2012, Projet Master automne (9 ét.)",3
"Informatique, 2010-2011, Projet Master automne (3 ét.)",3
"Informatique, 2008-2009, Projet Master automne (4 ét.)",3


In [502]:
data_master_summary_= pd.DataFrame(preview[1]) # this shows the number of times a name appears

data_master_summary_

,Nom Prénom
Bécholey Alexandre,4
Marx Clément,4
Brutsche Florian,4
Brot Benoît,4
Gruner Samuel,4
Kostic Teodora,4
Stewart Conail,4
Rousseau Adrien Jean-Louis,3
Nötzli Andres,3
El Khoury Raphael,3


In [504]:
data_master_sciper= pd.DataFrame(preview[9]) # this shows the number of times a particular sciper appears

data_master_sciper.head()

,No Sciper
170235,4
146742,4
159852,4
160213,4
173527,4


In [474]:
test = data_master.set_index("No Sciper")  # this is done to avoid comflict as no tw persons have same sciper

In [395]:
#Select needed Column
test_first = test[['No Sciper','Civilité','Nom Prénom']]


In [396]:
# set sciper as index since it is unique
test_first=test_first.set_index('No Sciper')



In [402]:
test_first.fillna('Nil') # May be useful

,Civilité,Nom Prénom
No Sciper,,
NaN,"Informatique, 2016-2017, Bachelor semestre 1 ...",Nil
No Sciper,Civilité,Nom Prénom
235688,Monsieur,Abbey Alexandre
274015,Monsieur,Ahn Seongho
268410,Madame,Alemanno Sara
271464,Monsieur,Althaus Luca
274518,Monsieur,Assi Karim
249613,Monsieur,Badoux Luc-Antoine
262214,Monsieur,Bagnoud Jérôme


In [443]:
test = ["Pro"]   #test = ["Ba", "Ma","Min","Mis","Pro","Sem","Sta"]
parameter_period_Master_pro = {};
for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith(tuple(test)):
            parameter_period_Master_pro[key]=value
print(parameter_period_Master_pro)




{'Projet Master printemps': '3781783', 'Projet Master automne': '249127'}


In [455]:
data_master2 = pd.DataFrame()
for value_section in parameter_section.values():
    for value_years in parameter_years.values():
        for value_period in parameter_period_Master_pro.values():
            for value_semester in parameter_semester.values():
                  #We use a try to prevent the problem when there is a empty list.
                try:
                    #The url is changed for every values in each loops
                    student_data = pd.read_html('http://isa.epfl.ch/imoniteur_ISAP//!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=%s&ww_x_PERIODE_ACAD=%s&ww_x_PERIODE_PEDAGO=%s&ww_x_HIVERETE=%s' % (value_section, value_years, value_period, value_semester),header=None)
                    
                    #The data is reshaped to be more suite for subsequent analysis
                    student_data.columns = student_data.loc[[1]].values.reshape(12,)
                    student_data.drop(student_data.index[[0,1]],inplace=True)
                    student_data = student_data[['Civilité','Nom Prénom',]]
                except:
                    pass
                
                #Each block of data is concatenated to have a unique dataframe
                data_master2 = pd.concat([data_master2, student_data[0]],axis=0)
                

In [456]:
data_master2.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
2,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
3,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
4,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN


In [457]:
#columns settings
data_master2.columns =  ["Civilité","Nom Prénom","Orientation Bachelor" ,"Orientation Master","Specialisation","Filière opt.","Mineur","Statut","Type Echange","Ecole Echange","No Sciper","NaN"]

In [458]:

data_master2= data_master2.drop(["Orientation Bachelor"], 1)



In [459]:
#fix the column head 
data_master2=data_master2.drop(data_master2.index[[1]])
data_master2

,Civilité,Nom Prénom,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
3,Monsieur,Brutsche Florian,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
4,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN
5,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,Stage,NaN,NaN,172264,NaN
0,"Informatique, 2011-2012, Projet Master printem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Monsieur,Bloch Marc-Olivier,NaN,NaN,NaN,NaN,Présent,NaN,NaN,178553,NaN
3,Monsieur,Bricola Jean-Charles,NaN,"Signals, Images and Interfaces",NaN,NaN,Stage,NaN,NaN,180731,NaN
4,Monsieur,Chakrabarty Soumitro,NaN,NaN,NaN,NaN,Stage,NaN,NaN,199654,NaN
5,Monsieur,Dosmukhamedov Diyar,NaN,Internet computing,NaN,NaN,Présent,NaN,NaN,192861,NaN


In [481]:
#fix index numbering
data_master2.index = range(len(data_master2.index))
data_master2

,Civilité,Nom Prénom,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
2,Monsieur,Brutsche Florian,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
3,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN
4,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,Stage,NaN,NaN,172264,NaN
5,"Informatique, 2011-2012, Projet Master printem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Monsieur,Bloch Marc-Olivier,NaN,NaN,NaN,NaN,Présent,NaN,NaN,178553,NaN
7,Monsieur,Bricola Jean-Charles,NaN,"Signals, Images and Interfaces",NaN,NaN,Stage,NaN,NaN,180731,NaN
8,Monsieur,Chakrabarty Soumitro,NaN,NaN,NaN,NaN,Stage,NaN,NaN,199654,NaN
9,Monsieur,Dosmukhamedov Diyar,NaN,Internet computing,NaN,NaN,Présent,NaN,NaN,192861,NaN


In [485]:
#create copy
test2 = data_master2.set_index("No Sciper")  # this is done to avoid comflict as no tw persons have same sciper

test2

,Civilité,Nom Prénom,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,NaN
No Sciper,,,,,,,,,,
NaN,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180027,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,Stage,NaN,NaN,NaN
159852,Monsieur,Brutsche Florian,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,NaN
166805,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,Stage,NaN,NaN,NaN
172264,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,Stage,NaN,NaN,NaN
NaN,"Informatique, 2011-2012, Projet Master printem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178553,Monsieur,Bloch Marc-Olivier,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
180731,Monsieur,Bricola Jean-Charles,NaN,"Signals, Images and Interfaces",NaN,NaN,Stage,NaN,NaN,NaN
199654,Monsieur,Chakrabarty Soumitro,NaN,NaN,NaN,NaN,Stage,NaN,NaN,NaN


In [491]:
# set sciper as index since it is unique
test_sec = test2[['Civilité','Nom Prénom']]


In [492]:
test_sec

,Civilité,Nom Prénom
No Sciper,,
NaN,"Informatique, 2008-2009, Projet Master automne...",NaN
180027,Madame,Agarwal Megha
159852,Monsieur,Brutsche Florian
166805,Monsieur,Fleury Marc-Olivier
172264,Monsieur,Garg Nikhil
NaN,"Informatique, 2011-2012, Projet Master printem...",NaN
178553,Monsieur,Bloch Marc-Olivier
180731,Monsieur,Bricola Jean-Charles
199654,Monsieur,Chakrabarty Soumitro


# Minor Master table

In [516]:
test = ["Min"]   #test = ["Ba", "Ma","Min","Mis","Pro","Sem","Sta"]
parameter_period_Master_min = {};
for key,value in dic_name_value.items():
    if key is not None: #checking for none because it exist in the keys. With this no error will be returned due to nonetype
        if key.startswith(tuple(test)):
            parameter_period_Master_min[key]=value
print(parameter_period_Master_min)




{'Mineur semestre 1': '2335667', 'Mineur semestre 2': '2335676'}


In [517]:
data_master_min = pd.DataFrame()
for value_section in parameter_section.values():
    for value_years in parameter_years.values():
        for value_period in parameter_period_Master_pro.values():
            for value_semester in parameter_semester.values():
                  #We use a try to prevent the problem when there is a empty list.
                try:
                    #The url is changed for every values in each loops
                    student_data = pd.read_html('http://isa.epfl.ch/imoniteur_ISAP//!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=%s&ww_x_PERIODE_ACAD=%s&ww_x_PERIODE_PEDAGO=%s&ww_x_HIVERETE=%s' % (value_section, value_years, value_period, value_semester),header=None)
                    
                    #The data is reshaped to be more suite for subsequent analysis
                    student_data.columns = student_data.loc[[1]].values.reshape(12,)
                    student_data.drop(student_data.index[[0,1]],inplace=True)
                    student_data = student_data[['Civilité','Nom Prénom',]]
                except:
                    pass
                
                #Each block of data is concatenated to have a unique dataframe
                data_master_min = pd.concat([data_master_min, student_data[0]],axis=0)
                

In [518]:
data_master_min.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
2,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
3,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
4,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN


In [519]:
data_master_min.columns =  ["Civilité","Nom Prénom","Orientation Bachelor" ,"Orientation Master","Specialisation","Filière opt.","Mineur","Statut","Type Echange","Ecole Echange","No Sciper","NaN"]

In [522]:
data_master_min=data_master_min.drop(data_master2.index[[1]])


data_master_min.head()

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
3,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
4,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN
5,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,172264,NaN


In [524]:
data_master_min.index = range(len(data_master_min))

data_master_min.head()


,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
0,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,180027,NaN
2,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,159852,NaN
3,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,166805,NaN
4,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,172264,NaN


In [527]:
test3 = data_master_min.set_index("No Sciper")  # this is done to avoid comflict as no tw persons have same sciper

test3.head()

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Specialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,NaN
No Sciper,,,,,,,,,,,
NaN,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180027,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,NaN
159852,Monsieur,Brutsche Florian,NaN,NaN,Internet computing,NaN,NaN,Congé,NaN,NaN,NaN
166805,Monsieur,Fleury Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,NaN
172264,Monsieur,Garg Nikhil,NaN,NaN,NaN,NaN,NaN,Stage,NaN,NaN,NaN


In [528]:
test_third = test3[['Civilité','Nom Prénom',"Specialisation"]]

test_third.head()

,Civilité,Nom Prénom,Specialisation
No Sciper,,,
NaN,"Informatique, 2008-2009, Projet Master automne...",NaN,NaN
180027,Madame,Agarwal Megha,NaN
159852,Monsieur,Brutsche Florian,Internet computing
166805,Monsieur,Fleury Marc-Olivier,NaN
172264,Monsieur,Garg Nikhil,NaN
